<a href="https://colab.research.google.com/github/fabienMoutarde/DLcourse/blob/master/Practical_CNN_Introduction_LeNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook has been prepared by Hsiu-Wen (Kelly) Chang from MINES ParisTech.

It adapts/combines notebooks from : [Fabien Moutarde and Guillaume Devineau] (https://github.com/fabienMoutarde/DLcourse/blob/master/tp2_Practical_CNN_Introduction_LeNet_MNIST.ipynb)

NOTE: For the colab version of the notebook please make sure that you are using python3 and a **GPU accelerated backend** (select *GPU* as Hardware accelerator either in the *Edit > Notebook settings* or in the *Runtime > Change runtime type* menus for the english colab interface, or in the *Execution > Modifier le type d'exécution* menus for the french colab interface).

## 0. Configuration

In [ ]:
import sys
print('Your python version: {}'.format(sys.version_info.major))
# Uncomment lines below only if you need them 
#!{sys.executable} -m pip install -U numpy --user
#!{sys.executable} -m pip install -U matplotlib --user
#!{sys.executable} -m pip install -U keras --user
#!{sys.executable} -m pip install -U tensorflow --user
#!{sys.executable} -m pip install -U theano --user

In [ ]:
USE_TENSORFLOW_AS_BACKEND = True
FORCE_CPU = False

In [ ]:
%matplotlib inline
import os
import numpy as np
from numpy import newaxis
import matplotlib.pyplot as plt

In [ ]:
if USE_TENSORFLOW_AS_BACKEND:
    os.environ['KERAS_BACKEND'] = 'tensorflow'
else:
    os.environ['KERAS_BACKEND'] = 'theano'
if FORCE_CPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
import keras
from keras import backend as K
from keras import regularizers
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model, Sequential # basic class for specifying and training a neural network
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

# Deep-Learning: Introduction to Convolutional Neural Networks

## 1. Introduction

Welcome to a tutorial to get you quickly up to speed with *deep learning*; from first principles, all the way to discussions of some of the intricate details, with the purposes of achieving respectable performance on one established machine learning benchmark: [MNIST](http://yann.lecun.com/exdb/mnist/) (classification of handwritten digits).


MNIST dataset                      
![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/mnist.png)  

By the end of this part of the tutorial, you should be capable of understanding and producing a simple CNN (with a structure similar to LeNet architecture) in Keras, achieving a respectable level of accuracy on MNIST.


In [ ]:
# --------------------------------------
# Dataset variables
# --------------------------------------
num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000  # there are 10000 test examples in MNIST
height, width, depth = 28, 28, 1 # MNIST images are 28x28 with 1 channel (greyscale)
num_classes = 10                 # there are 10 classes (1 per digit)

# --------------------------------------
# Dataset loading & preprocessing
# --------------------------------------
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

X_train = X_train[:, newaxis, :, :] # Reshape in "convolutionable" format (add tensor dim for the depth)
X_test = X_test[:, newaxis, :, :] # Reshape in "convolutionable" format (add tensor dim for the depth)

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

print(X_train.shape, Y_train.shape)


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(X_train[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(y_train[i]))
  plt.xticks([])
  plt.yticks([])
fig

## 2. Convolutions

It turns out that there is a very efficient way of pulling this off, and it makes advantage of the structure of the information encoded within an image---it is assumed that pixels that are spatially *closer* together will "cooperate" on forming a particular feature of interest much more than ones on opposite corners of the image. Also, if a particular (smaller) feature is found to be of great importance when defining an image's label, it will be equally important if this feature was found anywhere within the image, regardless of location.

Enter the **convolution** operator. Given a two-dimensional image, $\bf I$, and a small matrix, $\bf K$ of size $h \times w$, (known as a *convolution kernel*), which we assume encodes a way of extracting an interesting image feature, we compute the convolved image, ${\bf I} * {\bf K}$, by overlaying the kernel on top of the image in all possible ways, and recording the sum of elementwise products between the image and the kernel:

$$({\bf I} * {\bf K})_{xy} = \sum_{i=1}^h \sum_{j=1}^w {{\bf K}_{ij} \cdot {\bf I}_{x + i - 1, y + j - 1}}$$

(in fact, the exact definition would require us to flip the kernel matrix first, but for the purposes of machine learning it is irrelevant whether this is done)

The images below show a diagrammatical overview of the above formula and the result of applying convolution (with two separate kernels) over an image, to act as an edge detector:

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/convolve.png)
![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/lena.jpg)

## 3. Convolutional layers and Pooling layers

The convolution operator forms the fundamental basis of the **convolutional** layer of a CNN. The layer is completely specified by a certain number of kernels, $\bf \vec{K}$ (along with additive biases, $\vec{b}$, per each kernel), and it operates by computing the convolution of the output images of a previous layer with each of those kernels, afterwards adding the biases (one per each output image). Finally, an activation function, $\sigma$, may be applied to all of the pixels of the output images. Typically, the input to a convolutional layer will have $d$ *channels* (e.g. red/green/blue in the input layer), in which case the kernels are extended to have this number of channels as well, making the final formula of a single output image channel of a convolutional layer (for a kernel ${\bf K}$ and bias $b$) as follows:

$$\mathrm{conv}({\bf I}, {\bf K})_{xy} = \sigma\left(b + \sum_{i=1}^h \sum_{j=1}^w \sum_{k=1}^d {{\bf K}_{ijk} \cdot {\bf I}_{x + i - 1, y + j - 1, k}}\right)$$

Note that, since all we're doing here is addition and scaling of the input pixels, the kernels may be learned from a given training dataset via *gradient descent*, exactly as the weights of an MLP. In fact, an MLP is perfectly capable of replicating a convolutional layer, but it would require a lot more training time (and data) to learn to approximate that mode of operation.

Finally, let's just note that a convolutional operator is in no way restricted to two-dimensionally structured data: in fact, most machine learning frameworks ([Keras included](https://keras.io/layers/convolutional/)) will provide you with out-of-the-box layers for 1D and 3D convolutions as well!

It is important to note that, while a convolutional layer significantly decreases the number of *parameters* compared to a fully connected (FC) layer, it introduces more **hyperparameters**---parameters whose values need to be chosen *before* training starts.

Namely, the hyperparameters to choose within a single convolutional layer are:
- *depth*: how many different kernels (and biases) will be convolved with the output of the previous layer;
- *height* and *width* of each kernel;
- *stride*: by how much we shift the kernel in each step to compute the next pixel in the result. This specifies the overlap between individual output pixels, and typically it is set to $1$, corresponding to the formula given before. Note that larger strides result in smaller output sizes.
- *padding*: note that convolution by any kernel larger than $1\times 1$ will *decrease* the output image size---it is often desirable to keep sizes the same, in which case the image is sufficiently padded with zeroes at the edges. This is often called *"same"* padding, as opposed to *"valid"* (no) padding. It is possible to add arbitrary levels of padding, but typically the padding of choice will be either same or valid.

As already hinted, convolutions are not typically meant to be the sole operation in a CNN (although there have been promising recent developments on [all-convolutional networks](https://arxiv.org/pdf/1412.6806v3.pdf)); but rather to extract useful features of an image prior to downsampling it sufficiently to be manageable by an MLP.

A very popular approach to downsampling is a *pooling* layer, which consumes small and (usually) disjoint chunks of the image (typically $2\times 2$) and aggregates them into a single value. There are several possible schemes for the aggregation---the most popular being **max-pooling**, where the maximum pixel value within each chunk is taken. A diagrammatical illustration of $2\times 2$ max-pooling is given below.

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/pool.png)

## 4. Convolutional Neural Network (CNN) architecture

Now that we got all the building blocks, let's see what a typical convolutional neural network might look like!

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/cnn.png)

A typical CNN architecture for a $k$-class image classification can be split into two distinct parts---a chain of repeating $\mathrm{Conv}\rightarrow\mathrm{Pool}$ layers (sometimes with more than one convolutional layer at once), followed by a few fully connected layers (taking each pixel of the computed images as an independent input), culminating in a $k$-way softmax layer, to which a cross-entropy loss is optimised. I did not draw the activation functions here to make the sketch clearer, but do keep in mind that typically after every convolutional or fully connected layer, an activation (e.g. ReLU) will be applied to all of the outputs.

Note the effect of a single $\mathrm{Conv}\rightarrow\mathrm{Pool}$ pass through the image: it reduces height and width of the individual channels in favour of their number, i.e. *depth*.

The softmax layer and cross-entropy loss are both introduced in more detail [in the previous tutorial](http://online.cambridgecoding.com/notebooks/cca_admin/deep-learning-for-complete-beginners-recognising-handwritten-digits). For summarisation purposes, a softmax layer's purpose is converting any vector of real numbers into a vector of *probabilities* (nonnegative real values that add up to 1). Within this context, the probabilities correspond to the likelihoods that an input image is a member of a particular class. Minimising the cross-entropy loss has the effect of maximising the model's confidence in the *correct* class, without being concerned for the probabilites for other classes---this makes it a more suitable choice for probabilistic tasks compared to, for example, the squared error loss.

## 5. Detour: Overfitting, regularisation and dropout

This will be the first (and hopefully the only) time when I will divert your attention to a seemingly unrelated topic. It regards a very important pitfall of machine learning---**overfitting** a model to the training data. While this is primarily going to be a major topic of the next tutorial in the series, the negative effects of overfitting will tend to become quite noticeable on the networks like the one we are about to build, and we need to introduce a way to properly protect ourselves against it, before going any further. Luckily, there is a very simple technique we can use.

Overfitting corresponds to adapting our model to the training set to such extremes that its generalisation potential (performance on samples outside of the training set) is *severely* limited. In other words, our model might have learned the training set (along with any noise present within it) perfectly, but it has failed to capture the underlying process that generated it. To illustrate, consider a problem of fitting a sine curve, with white additive noise applied to the data points: 

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/plotsin.png)

Here we have a training set (denoted by blue circles) derived from the original sine wave, along with some noise. If we fit a degree-3 polynomial to this data, we get a fairly good approximation to the original curve. Someone might argue that a degree-14 polynomial would do better; indeed, given we have 15 points, such a fit would *perfectly* describe the training data. However, in this case, the additional parameters of the model cause catastrophic results: to cope with the inherent noise of the data, anywhere except in the closest vicinity of the training points, our fit is completely off.

Deep convolutional neural networks have a large number of parameters, especially in the fully connected layers. Overfitting might often manifest in the following form: if we don't have sufficiently many training examples, a small group of neurons might become responsible for doing most of the processing and other neurons becoming redundant; or in the other extreme, some neurons might actually become detrimental to performance, with several other neurons of their layer ending up doing nothing else but correcting for their errors.

To help our models generalise better in these circumstances, we introduce techniques of *regularisation*: rather than reducing the number of parameters, we impose *constraints* on the model parameters during training to keep them from learning the noise in the training data. The particular method I will introduce here is **dropout**---a technique that initially might seem like "dark magic", but actually helps to eliminate exactly the failure modes described above. Namely, dropout with parameter $p$ will, within a single training iteration, go through all neurons in a particular layer and, with probability $p$, *completely eliminate them from the network throughout the iteration*. This has the effect of forcing the neural network to cope with *failures*, and not to rely on existence of a particular neuron (or set of neurons)---relying more on a *consensus* of several neurons within a layer. This is a very simple technique that works quite well already for combatting overfitting on its own, without introducing further regularisers. An illustration is given below.

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/drop.png)

## 6. Applying a deep CNN to MNIST

As this post's objective, we will implement a deep convolutional neural network---and apply it on the MNIST digit recognition classification task.


As already mentioned, a CNN will typically have more hyperparameters than an MLP. For the purposes of this tutorial, we will also stick to "sensible" hand-picked values for them, but do still keep in mind that later on I will introduce a more proper method for learning them.

The hyperparameters are:
- The *batch size*, representing the number of training examples being used simultaneously during a single iteration of the gradient descent algorithm;
- The number of *epochs*, representing the number of times the training algorithm will iterate over the entire training set before terminating\*;
- The *kernel sizes* in the convolutional layers;
- The *pooling size* in the pooling layers;
- The *number of kernels* in the convolutional layers;
- The *dropout probability* (we will apply dropout after each pooling, and after the fully connected layer);
- The *number of neurons* in the fully connected layer of the MLP.

\* **N.B. here I have set the number of epochs to 100, which might be undesirably slow if you do not have a GPU at your disposal (the convolution layers are going to pose a significant performance bottleneck in this case). You might wish to decrease the epoch count and/or numbers of kernels if you are going to be training the network on a CPU.**


Modelling time! Our network has an architecture similar to LeNet5 of LeCun (see figure below). It will consist of two consecutive groups of one `Convolution2D` followed by  a `MaxPooling2D` layer. After the first pooling layer, the number of kernels is rougly doubled (in line with the previously mentioned principle of sacrificing height and width for more depth). Afterwards, the output of the second pooling layer is flattened to 1D (via the `Flatten` layer), and passed through one or two fully connected (`Dense`) layers. ReLU activations will once again be used for all layers except the output dense layer, which will use a softmax activation (for purposes of probabilistic classification).

![](http://perso.mines-paristech.fr/fabien.moutarde/ES_MachineLearning/TP_convNets/lenet5.png)

To regularise our model, a `Dropout` layer is applied after each pooling layer, and after the first `Dense` layer. This is another area where Keras shines compared to other frameworks: it has an internal flag that automatically enables or disables dropout, depending on whether the model is currently used for training or testing.

The remainder of the model specification is the following:
- We use the *cross-entropy* loss function as the objective to optimise (as its derivation is more appropriate for probabilistic tasks);
- We use the [*Adam* optimiser for gradient descent](http://sebastianruder.com/optimizing-gradient-descent/);
- We report the *accuracy* of the model (as the dataset is balanced across the ten classes)\*;
- We hold out 10% of the data for validation purposes.

\* To get a feeling for why accuracy might be inappropriate for unbalanced datasets, consider an extreme case where 90% of the test data belongs to class $x$ (this could be, for example, the task of diagnosing patients for an extremely rare disease). In this case, a classifier that just outputs $x$ achieves a seemingly impressive accuracy of 90% on the test data, without really doing any learning/generalisation.

In [ ]:
# --------------------------------------
# Network params
# --------------------------------------
# Network params
batch_size = 32    # in each iteration, we consider 32 training examples at once
num_epochs = 20    # we iterate 20 times over the entire training set
kernel_size = 5    # we will use 5x5 kernels throughout
pool_size = 2      # we will use 2x2 pooling throughout
conv_depth_1 = 6   # we will initially have 6 kernels in first conv. layer...
conv_depth_2 = 16  # ...switching to 16 after the first pooling layer
drop_prob_1 = 0.   # dropout after pooling with probability 0.
drop_prob_2 = 0.   # dropout in the FC layer with probability 0.
hidden_size = 128  # the FC layer will have 128 neurons
weight_penalty = 0. # Factor for weights penalty

# --------------------------------------
# CNN network definition
# --------------------------------------
# inp = Input(shape=(1,height,width)) # greyscale --> depth==1
# model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model = Sequential()
# Conv [8] -> Pool (with dropout on the pooling layer)
# newer version of Keras change border_mode = padding
model.add( Conv2D(conv_depth_1, (kernel_size,kernel_size), padding='same', activation='relu', data_format="channels_first", input_shape=( 1, 28, 28)) )
model.add( MaxPooling2D(pool_size=(pool_size, pool_size)) )
model.add( Dropout(drop_prob_1) )

# Conv [16] -> Pool (with dropout on the pooling layer)
model.add( Conv2D(conv_depth_2, (kernel_size,kernel_size), padding='same', activation='relu') )
model.add( MaxPooling2D(pool_size=(pool_size, pool_size)) )
model.add( Dropout(drop_prob_1) )

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
model.add( Flatten() )
model.add( Dense(hidden_size, activation='relu', kernel_regularizer=regularizers.l2(weight_penalty)) )
model.add( Dropout(drop_prob_2) )
model.add( Dense(num_classes, activation='softmax') )

print(model.summary())

This model, possibly after some tweaking of its architectural parameters, should be able to break $99\%$ accuracy on its **test set** with little to no effort.


## 7. Conclusion

Throughout this post we have covered the essentials of convolutional neural networks, introduced the problem of overfitting, and made a very brief dent into how it could be rectified via regularisation (by applying dropout) and successfully implemented a two-layer deep CNN (with LeNet like architecture) in Keras, applying it to MNIST, all in under 50 lines of code. 


## 8. Now lets train this model!

This code assume your already have installed <code>numpy</code>, <code>keras</code>, <code>tensorflow</code>, ... on your computer. If not, uncomment the adequate lines in the cell below:

In [ ]:
# --------------------------------------
# Loss function and Optimizer
# --------------------------------------
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

# --------------------------------------
# Training
# --------------------------------------
# newer version of keras change nb_epoch = epochs
history = model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.4) # ...holding out 40% of the data for validation

# --------------------------------------
# Evaluation on TEST set 
# --------------------------------------
for loss_name, loss_value in list(zip(model.metrics_names, model.evaluate(X_test, Y_test, verbose=1))):
    print('The final {} on the test set is: {:.2f}.'.format(loss_name, loss_value)) # Evaluate the trained model on the test set!

**Plot the training history, in order to check if overfitting seems to have occured or not, and if more training epochs could be performed.**


In [ ]:
# --------------------------------------
# Loss functions evolution
# --------------------------------------
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.legend(['train', 'validation'], loc='right')
plt.show()

# --------------------------------------
# Accuracies evolution
# --------------------------------------
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy by epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid()
plt.legend(['train', 'validation'], loc='right')
plt.show()


**You should see that over-fitting begins to occur.
Question 1: WHAT IS THE SIGN OF OVER-FITTING ON ABOVE GRAPH?
Answer: **

**There are 2 different methods that can be used to prevent over-fitting.
Question 2: WHAT ARE THOSE 2 OVER-FITTING PREVENTION METHODS?
Answer :**

**Test those 2 methods SEPARATELY below, by copy-pasting the original code, modifying it, and running it. **

## Assignment 1: 
in code cell below, by applying a good combination and parameterization of the 2 regularization techniques, + increasing the number of training epochs, + reducing the proportion of validation hold-out, TRY TO OBTAIN >99% acuracy ON TEST SET:

# Pre-trained Model: transfer Learning
In order not to spend the whole practical session staring at the screen while the network is training, we will not train the network from scratch but we will rather take a different approach called Transfer Learning.

When a model is trained on a given dataset for a given task, a knowledge useful for the task is accumulated. Transfer Learning is based on the hypothesis that this knowledge can -mostly- be reused for another similar dataset or task. In practice, this assumption is verified in a lot of cases.

** Transfer learning is partcularly powerful when re-using a convNet pre-trained for image classification on the huge and general ImageNet dataset. This is because it has learnt and "stored" in its lower layers a very GENERAL-PURPOSE transformation of images into a hierarchy of features. This learnt transformation is general enough to be used as a foundation onto which one can stack new layers for EASILY LEARNING ANY FUNCTION OF IMAGES: categorization for totally different classes, regression of camera pose, etc... Moreover, because the re-used layers transform input images into a very "pertinent" feature space, transfer learning make it possible to learn WITH VERY FEW EXAMPLES PER CLASS (and rather few epochs of training), as illustrated on the plot below.**


### Data download 

In [ ]:
# --- only if you are on your own laptop ---

# Uncomment to download the dataset
# ...on Mines ParisTech computers:
# !wget "http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz" 
# ... or on google colab:
![[ -f 101_ObjectCategories.tar.gz ]] || wget "http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz"

# Uncomment to extract the dataset
# ...on Mines ParisTech computers:
#!tar xzf 101_ObjectCategories.tar.gz
# ... or on google colab:
![[ -f 101_ObjectCategories ]] || tar xzf 101_ObjectCategories.tar.gz

# Display files
!ls

In [ ]:
# If you can't get the complete tar.gz file (unknown reason, please go to their website and download and decompressed), you can use your own google drive
from google.colab import auth
from google.colab import drive
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse
!python3 -m pip install colorama
 
# auth.authenticate_user()
drive.mount('/content/drive/')

## Data preprocessing

In [ ]:
def load_batch_images(path, n_examples_train, n_examples_validation=2, max_imgs_by_class=30, nb_categories=4, resize_to_size=(224,224)):

    images_train = []
    labels_train = []
    images_validation = []
    labels_validation = []
    
    # Get categories
    categories_list = os.listdir(path)
    categories_list = [category for category in categories_list if category != '.DS_Store']
    if isinstance(nb_categories, int) and len(categories_list) >= nb_categories:
        categories_list = categories_list[0:nb_categories]

    # For each category...
    for idx, category in enumerate(categories_list):
        # ... get images in category ...
        images_list = os.listdir(os.path.join(path, category))        
        if len(images_list) > max_imgs_by_class:
            images_list = images_list[:max_imgs_by_class]
        
        indx = 0
        for images in images_list[:n_examples_train + n_examples_validation]:                
            # ... load them ...
            img = load_img(os.path.join(path, category, images))  # as PIL images                             
            img = np.asarray(img) # as numpy array         
            img = Image.fromarray(img).resize(resize_to_size)  # resize
            if indx < n_examples_train:
                images_train.append(img)
                labels_train.append(idx)
            else:
                images_validation.append(img)
                labels_validation.append(idx)
            indx += 1
    
    # Convert to one-hot
    labels_train = np_utils.to_categorical(labels_train)
    labels_validation = np_utils.to_categorical(labels_validation)

    return images_train, labels_train, images_validation, labels_validation

In [ ]:
def shuffle_dataset(a, b):
    """Shuffle the entire dataset"""
    assert np.shape(a)[0] == np.shape(b)[0]
    p = np.random.permutation(np.shape(a)[0])
    return (a[p], b[p])

In [ ]:
from keras.preprocessing import image

def get_data(samples_by_category=8, dataset_path="./101_ObjectCategories/", nb_categories=4):

    images_train, labels_train, images_validation, labels_validation = load_batch_images(path=dataset_path, n_examples_train=samples_by_category, n_examples_validation=23, max_imgs_by_class=30, nb_categories=nb_categories)

    img_train = []
    img_validation = []

    for img in images_train:
        img = image.img_to_array(img)
        img =  preprocess_input(img)
        img_train.append(img)
    
    for img in images_validation:
        img = image.img_to_array(img)
        img =  preprocess_input(img)
        img_validation.append(img)

    if not USE_TENSORFLOW_AS_BACKEND:
        img_train = img_train.swapaxes(1, 3).swapaxes(2, 3)
        img_validation = img_validation.swapaxes(1, 3).swapaxes(2, 3)

    train = shuffle_dataset(np.asarray(img_train), labels_train)
    validation = shuffle_dataset(np.asarray(img_validation), labels_validation)

    images_train, labels_train = train
    images_validation, labels_validation = validation

    return images_train, labels_train, train, images_validation, labels_validation, validation

## Sanity check
Plot some image to make sure

In [ ]:
def demo_plot(images_path):
    demo_path = images_path
    #  demo_path = "/export/home/manitou/6/Commun/Neuronal/LSML/101_ObjectCategories/"
    demo_data, _, _, _ = load_batch_images(path=demo_path, n_examples_train=8, n_examples_validation=20)
    f, axarr = plt.subplots(2,2)
    axarr[0,0].imshow(demo_data[0])
    axarr[1,0].imshow(demo_data[1])
    axarr[0,1].imshow(demo_data[27])
    axarr[1,1].imshow(demo_data[31])
    axarr[0,0].grid('off')
    axarr[0,1].grid('off')
    axarr[1,0].grid('off')
    axarr[1,1].grid('off')
    plt.show()

## Data loader
To speed up the training, we will use only a few of the 101 classes.

In [ ]:
nb_categories = 22         # out of 101 classes
samples_by_category = 7  # out of about 50 images for each category
#images_path = './101_ObjectCategories/'
# Load the data!
train_img, train_labels, train, val_img, val_labels, val = get_data(samples_by_category=samples_by_category, dataset_path=images_path, nb_categories=nb_categories)

# Print the tensors shapes
print('Shapes (train) :', train_img[0].shape, train_labels[0].shape)

## Transfer Learning

### Load a pre-trained model

Create a neural network with the Inception architecture. Instead of learning the weights, we use existing weights obtained by (pre-)training the network on a dataset named Imagenet [2] for a classification task. Imagenet consists in 14,197,122 images associated with 1,000 classes, ranging from apples to lakes, radiators, or funambulists (http://image-net.org/explore?wnid=n00324978).

We also tweak the network.

Note the <code>keras</code> option <code>include_top=False</code>. On the one hand, one can view the first layers, which are close to the raw pixels input, as low-level signal processing units, since they mostly perform corner detection, shape detection or texture detection for instance. On the other hand, the last layers take already high-level representations of the input images and assign them to the desired classes.
We keep the first layers and their weights since re-training them will take time but will likely provide very limited value. The high-levels we will later use are more specific to our use case.

<code>[2] http://image-net.org</code>

In [ ]:
# create the base pre-trained model
# (the pretrained weights are stored in a file located at: https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5)
base_model = InceptionV3(weights='imagenet', include_top=False)

Note the parameters count, and how many are learnable:

In [ ]:
base_model.summary()

## Change the head of the pre-trained model

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(32, activation='relu')(x)

# and a logistic layer -- we have nb_categories classes
predictions = Dense(nb_categories, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)



** For now, let's not change (i.e. not train) the "knowledge" stored into weights during the pre-training on ImageNet. To this end, we "freeze" the weights of all pre-trained layers: only the final layers just added for adapting to our problem will be learnt:**

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# print adapted model
model.summary()

## Training

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# train the model on the new data for a few epochs
history = model.fit(train_img, train_labels, batch_size=32, validation_data=(val_img, val_labels), epochs=20)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss by epoch for %d samples per category' % samples_by_category)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='right')
plt.show()

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy by epoch for %d samples per category' % samples_by_category)
plt.ylabel('accuracy')
plt.legend(['train', 'validation'], loc='right')
plt.xlabel('epoch')
plt.show()

### Assignment 2
Try to reuse the code and generate the blue line in this figure
![](http://perso.mines-paristech.fr/fabien.moutarde/ES_LSML/TP_TransferLearning-ConvNet/transfer-learning_Caltech-veryFewExamplesPerClass.png) 
